# Исследование надежности заемщиков.

## Шаг 1. Обзор данных

Привет ревьюер! Думаю сразу перейдём на "ты", если не против)
Тут я себе небольшие вставки буду делать, надеюсь будешь не против.


Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Цель исследования:
1) Есть ли зависимость между количеством детей и возвратом кредита в срок?   
2) Есть ли зависимость между семейным положением и возвратом кредита в срок?   
3) Есть ли зависимость между уровнем дохода и возвратом кредита в срок?  
4) Как разные цели кредита влияют на его возврат в срок?   

ps. заранее думаю, что все 4 пункта влияют, но убедимся в этом наглядно

Начнём.
Данные от банка о статистике платёжеспособности клиентов получаем отсюда "/datasets/data.csv". Т.к. мы видим данные впервые, то сразу предполагаем о том, что данные неполные\неверные и тп.

Так что сначала проверяем данные на ошибки и оцениваем их влияние на исследование. Затем, на этапе предобработки исправляем данные до приемлемого для исследования

Таким образом, исследование пройдёт в четыре этапа:

Обзор данных.  
Предобработка данных.  
Проверка гипотез.  
Общий вывод.  


In [1]:
# Импортируем библитеки

import pandas as pd
import numpy as np
import pprint

from pymystem3 import Mystem
from collections import Counter

In [2]:
# загружаем файл с данными

data = pd.read_csv('/datasets/data.csv')

In [3]:
# Смотрим с начала и с конца данные

data.head(15)
#data.tail(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# Получаем информацию о данных

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Итак, мы посмотрели данные, что мы можем скзаать на первый взгляд.
В таблице имеется 12 столбцов:
* `children` - колличество детей;
* `days_employed` - трудовой стаж;
* `dob_years` - полных лет;
* `education` - уровень образования;
* `education_id` - id код уровня образования;
* `family_status` - семейное положение;
* `family_status_id` - id код семейного положения;
* `gender` - пол;
* `income_type` - тип занятости;
* `debt` -  имел ли задолженность по возврату кредитов;
* `total_income` -  ежемесячный доход;
* `purpose` - цель получения кредита.

В названиях столбцов нарушения стиля не наблюдается, всё единообразно - зачёт.
Однако в данных мы видим следующее:
1) В столбце `education` наблюдается верхний регистр;  
2) В столбцах `days_employed` и `total_income` наблюдаются пропуски NaN;  
3) В столбце `days_employed` имеются отрицательные значения;  
4) Данные столбца `total_income` лучше перевести в формат int (ps копейки не считаем😎)   

## Шаг 2. Предобработка данных

### Работа с пропусками и дупликатами

#### Работа с пропусками

Для начала посчитаем общее колличество пропусков

In [6]:
# Смотрим на количество пропусков

data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Из 21к значений, 2100 является 1\10, что уже досаточно серьёзно влияет на результаты исследования.
Причём данные столбцы обладают очень важной информацией, для принятия решения о даче\отказе выдачи кредита. По хорошему тут бы заполонить пропуски стохастической линейной регрессией, но я честно хз как её сюда вписать(Ps. точнее я увидел, как её писать в инете и как-то перехотел))) ). За неимением лучшего возьмём то, что предлагает программа обучения - медианное значение. В данном случае за неимением возможности получить пропущенные данные это будет наилучшим решением. Мода слишком завысит, а минимальное значение не отразит всю картину.

In [7]:
# Посмотрим у каких людей нет данных

data[data['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Судя по всему это люди "старшего" поколения, и для них среднее значение будет допустимо

#### Работа с дупликатами

In [8]:
# Проверим дупликаты

data.duplicated().sum()

54

Немного, но имеется, удалим же их и потом проверим, на результатх исследования такая небольшая выборка не должна сыграть роли. Появились они возможно из-за ошибки ввода данных

In [9]:
# Удаляем дупликаты

data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

### Работа с ошибками и аномалиями

In [10]:
# Проверяем столбцы на ошибки и аномалии

#data['education'].sort_values().unique() # Тут у нас разброс в регистрах, надо бы всё привести к единообразию
#data['family_status'].sort_values().unique() # Тут всё норм
#data['gender'].sort_values().unique() # XNA? Допустим - нет данных
#data['income_type'].sort_values().unique() # Тут всё норм
data['children'].sort_values().unique() # -1? 20? Тут явно ошибка

array([-1,  0,  1,  2,  3,  4,  5, 20])

In [11]:
# Исправляем ошибки и аномалии
# с детьми и с регистром разобрались
# заменяем 20 детей на среднее, ибо мы не знаем, что имел ввиду тот кто вводил данные, это 2 или 0 детей, возьмём среднее

data['children'] = data['children'].replace(-1, 1)
children_median = data.loc[data.loc[:, 'children'] != 20]['children'].median()
data['children'] = data['children'].replace(20, children_median)
data['education'] = data['education'].str.lower()

In [12]:
data['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

Начнём заполнение пропусков со столбца `days_employed` т.к. в нём имеются отрицательные значения, из-за чего медиальное значение будет иметь слишком большую вилку\разброс.

In [13]:
# Отбрасываем знак и проверяем, всё ли впорядке

data['days_employed'] = data['days_employed'].abs()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1.0,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0.0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3.0,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0.0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [14]:
#data['total_income'] = data['total_income'].astype('int') # ага, значит сначала необходимо убрать NaN, ошибся

# Заполняем пропуски
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1.0,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0.0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3.0,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0.0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0.0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0.0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0.0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2.0,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0.0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [15]:
data['dob_years'].describe()

count    21471.000000
mean        43.279074
std         12.574291
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [16]:
data['dob_years'] = data['dob_years'].replace(0, data['dob_years'].mean())

In [17]:
data['dob_years'].describe()

count    21471.000000
mean        43.482660
std         12.217199
min         19.000000
25%         33.500000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [18]:
data['days_employed'].describe()

count     21471.000000
mean      60524.366200
std      133393.041787
min          24.141633
25%        1024.274090
50%        2194.220567
75%        4794.911909
max      401755.400475
Name: days_employed, dtype: float64

In [19]:
filter = data['days_employed'] > 20000
data.loc[filter, ['days_employed']] =20000

In [20]:
data['days_employed'].describe()

count    21471.000000
mean      5168.777375
std       6780.533508
min         24.141633
25%       1024.274090
50%       2194.220567
75%       4794.911909
max      20000.000000
Name: days_employed, dtype: float64

In [21]:
# Вообще мне хочется так же изменить данные и в рабочих днях, заодно и памяти съкономлю, если скажешь убрать - уберу
# помимо прочего решил перевести трудовой стаж в года и удалить столбец с днями

data['years_employed'] = data['days_employed'] / 365 
del data['days_employed'] 
data['years_employed'] = data['years_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1.0,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1.0,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0.0,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3.0,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0.0,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,54


Таблица стала выглядеть попризентабельнее

### Работа с таблицей

In [22]:
# Теперь займёмся целями кредита

unique_purposes = data['purpose'].unique().tolist()
unique_purposes

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

Данные цели схожи, их лучше будет объединить в несколько категорий:
1) недвижимость
2) образование
3) свадьба
4) авто

In [23]:
# Приводим цели к 4 категориям
def purpose_a (purpose):
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'образова' in purpose:
        return 'получение образования'
    else:
        return 'операции с автомобилем'
data['purpose_category'] = data['purpose'].apply(purpose_a)

In [24]:
unique_purposes = data['purpose_category'].unique().tolist()
unique_purposes

['операции с недвижимостью',
 'операции с автомобилем',
 'получение образования',
 'проведение свадьбы']

**Промежуточный итог.**
Мы проверили таблицу вначале и конце, собрали о ней информацию. Затем мы посчитали количество пропусков, заполнили пропуски средним значением, удалили дупликаты(так как это скорее ошибка ввода). После чего мы изменили аномальные значения, привели таблицу к единообразию, и сделали её более удобной для чтения и понимания.
В таблице ещё остались аномальные значения, но их выборка слишком мала, чтобы повлиять на результаты исследования

## Шаг 3. Проверка гипотез

### 1) Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [25]:
children_pivot = data.pivot_table(index = ['children'], columns = ['debt'], values = 'purpose', aggfunc='count')

children_pivot['influence'] = children_pivot[1] / (children_pivot[0]+children_pivot[1]) 
children_pivot

debt,0,1,influence
children,,,
0.0,13112.0,1071.0,0.075513
1.0,4411.0,445.0,0.091639
2.0,1858.0,194.0,0.094542
3.0,303.0,27.0,0.081818
4.0,37.0,4.0,0.097561
5.0,9.0,NaN,NaN


Данные несколько неоднозначны, однако по ним можно сделать вывод, что люди без детей чаще выплачивают в срок

### 2) Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
family_pivot = data.pivot_table(index = ['family_status'], columns = ['debt'], values = 'total_income', aggfunc='count')

family_pivot['influence'] = family_pivot[1] / (family_pivot[0] + family_pivot[1])
family_pivot

debt,0,1,influence
family_status,,,
Не женат / не замужем,2536,274,0.097509
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693
гражданский брак,3775,388,0.093202
женат / замужем,11413,931,0.075421


А тут результат очевиден. Люди вне брака\в гражданском браке(скорее всего молодые) хуже погашают кредиты. Возможно из-за меньшей "ответственности" данной категории людей. 
Самые низкие показатели у разведённых и потерявших супруга\у, скорее всего данная категория граждан на опыте ведения бюджета более ответственно подходят к финансам.

### 3) Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
def total_income_category(total_income):
    if total_income <= 30000:
            return 'Низкий уровень дохода (E)'
    if total_income <= 50000:
            return 'Средний уровень дохода (D)'
    if total_income <= 200000:
            return 'Выше среднего уровень дохода (C)'
    if total_income <= 1000000:
            return 'Высокий уровень дохода (В)'
    return 'Сверхвысокий уровень дохода (А)'

In [28]:
data['total_income_category'] = data['total_income'].apply(total_income_category)
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,purpose_category,total_income_category
0,1.0,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23,операции с недвижимостью,Высокий уровень дохода (В)
1,1.0,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11,операции с автомобилем,Выше среднего уровень дохода (C)
2,0.0,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,операции с недвижимостью,Выше среднего уровень дохода (C)
3,3.0,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11,получение образования,Высокий уровень дохода (В)
4,0.0,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,54,проведение свадьбы,Выше среднего уровень дохода (C)


In [29]:
income_pivot = data.groupby('total_income_category')['debt'].sum() / \
data.groupby('total_income_category')['debt'].count() * 100

In [30]:
income_pivot.sort_values()

total_income_category
Средний уровень дохода (D)          6.000000
Высокий уровень дохода (В)          7.062091
Сверхвысокий уровень дохода (А)     8.000000
Выше среднего уровень дохода (C)    8.482505
Низкий уровень дохода (E)           9.090909
Name: debt, dtype: float64

На основании данных результатов можно сделать вывод, что уровень дохода влияет на возврат кредита в срок. По таблице можно сказать, что самыми надёжными с точки зрения возврата кредита является люди со средним и высоким уровнем дохода. Самыми ненадёжными являются люди с низкими доходами. Однако удивителен факт наличия людей категории А и С среди возможных должников по кредиту. Если людей категории А можно объяснить большим колличеством операций с покупками и кредитами, и человек может забывать оплачивать кредит вовремя, то появление категории С объяснить сложно.

### 4) Как разные цели кредита влияют на его возврат в срок?

In [31]:
purpose_pivot = data.pivot_table(index=['purpose_category'], columns=['debt'], values='education_id', aggfunc='count')
purpose_pivot['influence'] = purpose_pivot[1] / (purpose_pivot[0] + purpose_pivot[1])
purpose_pivot

debt,0,1,influence
purpose_category,,,
операции с автомобилем,3905,403,0.093547
операции с недвижимостью,10032,782,0.072314
получение образования,3644,370,0.092177
проведение свадьбы,2149,186,0.079657


По результатам исследования ненадёжными людьми будут люди получающие образование или покупающие авто. Это можно объяснить молодым возрастом данной группы лиц, когда они ещё не "встали на ноги". Операции с недвижимостью проходят у людей, уже имеющие стабильный доход, и предположительно зачастую семью, что делает их наиболее ответственной категорией людей. Однако кредит на свадьбу показывает ниже ожиданий результаты. Ибо данное мероприятие не относится никак к инвестициям в будущее, а является грубо говоря "проеданием" средств.

## Шаг 4. Общий вывод и рекомендации

### Общие выводы

На основании исследования получилось подтвердить все выдвинутые гипотезы: 

**1) Зависимость между количеством детей и возвратом кредита в срок**  
Данные не однозначные, возможно, нужна большая выборка, чем та, которую мы имеем. Бездетные, как правило реже просрачивают оплату по кредиту, чем люди с детьми.

**2) Зависимость между семейным положением и возвратом кредита в срок**  
Зависимость есть, люди вне брака или в пребывающие в гражданском браке чаще не платят кредит вовремя.

**3) Зависимость между уровнем дохода и возвратом кредита в срок**  
Зависимость есть, наиболее надёжными являются люди среднего достатка, а вот люди низкого достатка являются ненадёжными.

**4) Влияют ли цели кредита на его возврат в срок**  
Зависимость есть, наиболее надёжными будут люди которые берут кредит на недвижимость и свадьбу, менее для покупки авто или образования.


### Рекомендации

Самыми надёжными с точки зрения данных будут люди пребывающие или пребывавшие в браке, без детей, либо с 3 детьми, обладающими средним достатком и целью кредита которых является покупка недвижимости.
Из ненадёжным заёмщиком будет человек вне брака, с детьми, обладающим низким достатком, целью кредита которого является покупка авто. Зачастую это будут молодые люди.

**Рекомендации для банка**  
- При получении данных спрашивать рабочий стаж в годах, а не днях; 
- Не отправлять данные с заведомо аномальными или ошибочными данными, которые сильно влияют на статистику(доходы, количество детей).
